# Engenharia de Atributos

Importando as dependências

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Lendo os dados brutos e concatenando-os em um único dataframe

In [43]:
base_path = '../data/raw/' 

dados_2021 = pd.read_csv(base_path + '2021.csv', encoding='latin-1', sep=';')
dados_2022 = pd.read_csv(base_path + '2022.csv', sep=',')
dados_2023 = pd.read_csv(base_path + '2023.csv', encoding='latin-1', sep=';')
dados_2024 = pd.read_csv(base_path + '2024.csv', sep=',')

dados = pd.concat([dados_2021, dados_2022, dados_2023, dados_2024], axis=0, ignore_index=True)

dados.shape

(250889, 30)

Na análise exploratória, foram indentificadas colunas em que haviam dados nulos:

In [44]:
nulos = dados.isnull().sum().sort_values(ascending=False)
nulos[nulos>0]

uop                       219
delegacia                  85
regional                   23
classificacao_acidente      4
dtype: int64

Reconhecendo os dados dessas colunas

In [45]:
dados['uop'].head(5), dados['delegacia'].head(5), dados['regional'].head(5), dados['classificacao_acidente'].head(5)

(0    UOP01-DEL05-SP
 1    UOP01-DEL01-AM
 2    UOP03-DEL04-CE
 3    UOP02-DEL04-RJ
 4    UOP04-DEL05-RJ
 Name: uop, dtype: object,
 0    DEL05-SP
 1    DEL01-AM
 2    DEL04-CE
 3    DEL04-RJ
 4    DEL05-RJ
 Name: delegacia, dtype: object,
 0    SPRF-SP
 1    SPRF-AM
 2    SPRF-CE
 3    SPRF-RJ
 4    SPRF-RJ
 Name: regional, dtype: object,
 0    Com Vítimas Feridas
 1                    NaN
 2    Com Vítimas Feridas
 3    Com Vítimas Feridas
 4    Com Vítimas Feridas
 Name: classificacao_acidente, dtype: object)

Substituindo os dados faltantes pela moda das respectivas colunas, mas considerando a unidade federativa específica para a substituição assim os dados ficarão mais precisos

In [46]:
for index, row in dados.iterrows():

    if pd.isnull(row['uop']):

        uf_especifica = row['uf'] 
        aux_df = dados[dados['uf'] == uf_especifica]
        if not aux_df['uop'].mode().empty:
            dados.at[index, 'uop'] = aux_df['uop'].mode()[0]

    if pd.isnull(row['delegacia']):

        uf_especifica = row['uf'] 
        aux_df = dados[dados['uf'] == uf_especifica]

        if not aux_df['delegacia'].mode().empty:
            dados.at[index, 'delegacia'] = aux_df['delegacia'].mode()[0]

    if pd.isnull(row['regional']):

        uf_especifica = row['uf'] 
        aux_df = dados[dados['uf'] == uf_especifica]

        if not aux_df['regional'].mode().empty:
            dados.at[index, 'regional'] = aux_df['regional'].mode()[0]

    if pd.isnull(row['classificacao_acidente']):

        uf_especifica = row['uf'] 
        aux_df = dados[dados['uf'] == uf_especifica]

        if not aux_df['classificacao_acidente'].mode().empty:
            dados.at[index, 'classificacao_acidente'] = aux_df['classificacao_acidente'].mode()[0]

Checando se os valores nulos foram realmente preenchidos. Como nenhuma coluna com valor nulo é impressa, não existem mais valores nulos na base.

In [47]:
nulos = dados.isnull().sum().sort_values(ascending=False)
nulos[nulos>0]


Series([], dtype: int64)

Tratamento de data/hora, mudando para o formato datetime padrão americano MM-DD-YYYY HH:MM AM/PM que geralmente são usados nos bancos de dados

In [48]:
dados['datetime'] = pd.to_datetime(dados['data_inversa'] + ' ' + dados['horario'])

# padrão americano, geralmente usado nos banco de dados
dados['datetime'] = dados['datetime'].dt.strftime('%m-%d-%Y %I:%M %p')

dados = dados.drop(columns=['data_inversa', 'horario'])

In [52]:
dados[['id', 'datetime']]

,id,datetime
0,331730.0,01-01-2021 05:30 AM
1,331804.0,01-01-2021 08:05 AM
2,331815.0,01-01-2021 10:10 AM
3,331823.0,01-01-2021 12:30 PM
4,331843.0,01-01-2021 02:40 PM
...,...,...
250884,635554.0,09-27-2024 04:15 PM
250885,635630.0,09-15-2024 08:30 PM
250886,635699.0,08-13-2024 04:22 PM
250887,635887.0,09-21-2024 06:50 PM
